In [2]:
!virtualenv env
!source env/bin/activate
!pip3 install nltk
!pip3 install numpy
!pip3 install pandas
!pip3 install num2words
!pip3 install tqdm

Using base prefix '/Library/Frameworks/Python.framework/Versions/3.8'
New python executable in /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/python3.8
Not overwriting existing python script /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/python (you must use /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/python3.8)
Installing setuptools, pip, wheel...
done.
Overwriting /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/activate with new content
Overwriting /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/activate.fish with new content
Overwriting /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/activate.csh with new content
Overwriting /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/activate.xsh with new content
Overwriting /Users/alexandermaxwell/school/fall2021/4206/final/env/bin/python-config with new content
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
import pandas as pd
from collections import Counter
import numpy as np
import json
from tqdm import tqdm

In [4]:
corpus = []
for line in open('combined.json', 'r'):
    corpus.append(json.loads(line))

In [5]:
pd_data = pd.DataFrame(corpus)
pd_data.head()
length = len(pd_data)
alpha = .3 #decreased weight factor for the body. this increases relevance of the title by a factor of 3

In [13]:
Titles = pd_data.title
Contents = pd_data.contents
display(pd_data)


,id,title,contents,date,topics,components
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"PORTLAND, Oregon. – Mohamed Osman Mohamud, 23,...",2014-10-01T00:00:00-04:00,[],[National Security Division (NSD)]
1,12-919,$1 Million in Restitution Payments Announced t...,WASHINGTON – North Carolina’s Waccamaw River...,2012-07-25T00:00:00-04:00,[],[Environment and Natural Resources Division]
2,11-1002,$1 Million Settlement Reached for Natural Reso...,BOSTON– A $1-million settlement has been...,2011-08-03T00:00:00-04:00,[],[Environment and Natural Resources Division]
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,WASHINGTON—A federal grand jury in Las Vegas...,2010-01-08T00:00:00-05:00,[],[Environment and Natural Resources Division]
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,[Environment],[Environment and Natural Resources Division]
...,...,...,...,...,...,...
13082,16-735,Yuengling to Upgrade Environmental Measures to...,The Department of Justice and the U.S. Environ...,2016-06-23T00:00:00-04:00,[Environment],[Environment and Natural Resources Division]
13083,10-473,Zarein Ahmedzay Pleads Guilty to Terror Violat...,The Justice Department announced that Zarein...,2010-04-23T00:00:00-04:00,[],[Office of the Attorney General]
13084,17-045,Zimmer Biomet Holdings Inc. Agrees to Pay $17....,Subsidiary Agrees to Plead Guilty to Violating...,2017-01-12T00:00:00-05:00,[Foreign Corruption],"[Criminal Division, Criminal - Criminal Fraud ..."
13085,17-252,ZTE Corporation Agrees to Plead Guilty and Pay...,ZTE Corporation has agreed to enter a guilty p...,2017-03-07T00:00:00-05:00,"[Asset Forfeiture, Counterintelligence and Exp...","[National Security Division (NSD), USAO - Texa..."


Pre-Processing of data for readability

In [6]:
def convert_lower_case(data):
    tokens = word_tokenize(str(data))
    temp = ""
    for word in tokens:
        temp = temp + " " + word.lower()
    return temp

In [7]:
def remove_punctuation(data):
    punctuation = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for char in punctuation:
        data = data.replace(char, " ")
    return data

In [8]:

def remove_stop_words(data):
    temp = ""
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(data)
    for word in tokens:
        if not word in stop_words:
            temp = temp + " " + word
    return temp

In [9]:
def stem(data):
    stemmer = PorterStemmer()

    tokens = word_tokenize(str(data))
    temp = list()
    for word in tokens:
        temp.append(stemmer.stem(word))
    return temp

In [10]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    temp = ""
    for word in tokens:
        try:
            word = num2words(int(word))
        except:
            pass
        temp = temp + " " + word
    return temp


In [11]:
def preprocess(data):
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    data = stem(data)
    return data

Processing of Data

In [14]:
Processed_Titles = list()
for item in tqdm(Titles):
    Processed_Titles.append(word_tokenize(str(preprocess(item))))

100%|██████████| 13087/13087 [00:16<00:00, 790.34it/s]


In [15]:
Processed_Contents = list()
for item in tqdm(Contents):
    Processed_Contents.append(word_tokenize(str(preprocess(item))))

100%|██████████| 13087/13087 [05:54<00:00, 36.93it/s] 


Calculating Document Frequency for Corpus

In [16]:
DF = {}

for i in range(length):
    tokens = Processed_Contents[i]
    for word in tokens:
        try:
            DF[word].add(i)
        except:
            DF[word] = {i}
    tokens = Processed_Titles[i]
    for word in tokens:
        try:
            DF[word].add(i)
        except:
            DF[word] = {i}

for i in DF:
    DF[i] = len(DF[i])





In [17]:
Vocab_Size = len(DF)
print(Vocab_Size)


46926


In [18]:
def Document_Frequency(word):
    freq = 0
    try:
        freq = DF[word]
    except:
        ...
    return freq

Calculate Document Frequency

In [19]:
def TF_IDF(Processed_Items):
    Temp_TF_IDF = {}
    for i in tqdm(range(length)):
        tokens = Processed_Contents[i]

        counter = Counter(tokens + Processed_Items[i])
        words_count = len(tokens + Processed_Items[i])

        for token in np.unique(tokens):
            tf = counter[token]/words_count
            df = Document_Frequency(token)
            idf = np.log((length+1)/(df+1)) #numerator is added 1 to avoid negative values
            Temp_TF_IDF[i, token] = tf*idf
            #print(f"tf {tf} df {df} idf {idf} ")
        
    return Temp_TF_IDF
    

In [20]:
Title_TF_IDF = TF_IDF(Processed_Titles)
Contents_TF_IDF = TF_IDF(Processed_Contents)

100%|██████████| 13087/13087 [00:15<00:00, 844.39it/s]


In [21]:
for value in Contents_TF_IDF:
    Contents_TF_IDF[value] *= alpha #correctly weighting the body for less than the title

Merging the two indexes

In [22]:
for value in Title_TF_IDF:
    Contents_TF_IDF[value] = Title_TF_IDF[value]

In [23]:
len(Contents_TF_IDF)

2665778

Matching Function using vectorized data

In [24]:

def matching_score(results, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in Contents_TF_IDF:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += Contents_TF_IDF[key]
            except:
                query_weights[key[0]] = Contents_TF_IDF[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:results]:
        l.append(i[0])
    
    return l 
    
    

In [25]:
query = input("what is your query? ")
n = 1
for doc in matching_score(5,query):
    print(f"article {n}")
    n+=1
    display(pd_data.iloc[doc].contents)

Matching Score

Query: shower

['[', "'shower", "'", ']']

article 1


"A federal jury late yesterday convicted Kevin Casaus, 25, a former corrections officer at the Bernalillo County, N.M., Metropolitan Detention Center (MDC) in Albuquerque, N.M., on obstruction of justice and falsification of records charges, announced the Justice Department. \xa0 Casaus and fellow former MDC corrections officers, Demetrio Juan Gonzales, 40, and Matthew Pendley, 26, were indicted in June 2012, and charged with various crimes related to the assault of an inmate housed at MDC on Dec. 21, 2011, and subsequent attempts to cover up and impede the investigation of the assault.\xa0 In Oct. 2012, Gonzales pleaded guilty to violating the civil rights of an individual in his custody when he struck and choked the victim in the shower room/dress out area of MDC.\xa0 Pendley pleaded guilty in Feb. 2012 to obstructing justice by making false statements to law enforcement during their investigation of the assault on an inmate.\xa0 \xa0 According to the evidence at trial, during the ea

article 2


'    Demetrio Juan Gonzales, 40, a former corrections officer at the Bernalillo County Metropolitan Detention Center (MDC) in Albuquerque, N.M., pleaded guilty today to one count of violating the civil rights of an individual in his custody when he struck and choked the victim in the shower room/dress out area of MDC.     According to court documents, during the early morning hours of Dec. 21, 2011, Gonzales was assigned to the Receiving-Discharge-Transfer (RDT) Unit at MDC where individuals are brought to be booked soon after they are arrested.\xa0 Gonzales’ job was to photograph and fingerprint those who are brought to RDT for booking.\xa0 The victim, who had been arrested for Driving While Intoxicated, was verbally uncooperative during the booking process, but was not a physical threat to anyone.\xa0 Nonetheless, Gonzales became angry at the victim and walked him to the shower room/dress out area where he knew there were no surveillance cameras.\xa0 Several other corrections officer

article 3


"The Justice Department announced today that Kevin Casaus, 24, a former corrections officer at the Bernalillo County Metropolitan Detention Center (MDC) in Albuquerque, N.M., was sentenced this morning to serve 15 months in federal prison followed by one year of supervised release for his conviction on obstruction of justice and falsification of records charges. \xa0 Casaus and fellow former MDC corrections officers, Demetrio Juan Gonzales, 41, and Matthew Pendley, 26, were indicted in June 2012, and charged with various crimes related to the Dec. 21, 2011 assault of an inmate housed at MDC, and subsequent attempts to cover up and impede the investigation of the assault.\xa0 \xa0 On March 6, 2013, a federal jury convicted Casaus on obstruction of justice and falsification of records charges, and acquitted him on a related assault charge.\xa0 According to the evidence at trial, during the early morning hours of Dec. 21, 2011, Gonzales was assigned to the Receiving-Discharge-Transfer (RD

article 4


'U.S. Attorney Ronald C. Machen Jr. of the District of Colombia and Chief Cathy L. Lanier of the Metropolitan Police Department announced that Bernard Freundel, a rabbi who had worked for a Jewish congregation in Washington, D.C., pleaded guilty today to 52 counts of voyeurism stemming from a series of incidents between 2009 and 2014 in which he secretly took video recordings of women preparing for a Jewish ritual bath. Freundel, 63, of Washington, D.C., pleaded guilty to the misdemeanor charges before the honorable Senior Judge Geoffrey M. Alprin in the superior court of the District of Columbia.\xa0 Each count carries a maximum sentence of a year of incarceration, a fine of up to $1,000 or $2,500, for the offenses which occurred on or after June 11, 2013, or both.\xa0 Freundel faces a maximum of 52 years in prison and the potential fines when he is sentenced on May 15, 2015. “Bernard Freundel exploited his position of power to victimize dozens of women who entered a sacred, intimate 

article 5


'Bernard “Barry” Freundel, 63, of Washington D.C., a rabbi who had worked for a Jewish congregation in Washington, D.C., was sentenced today to a prison term of six years and six months on 52 counts of voyeurism stemming from a series of incidents between 2009 and 2014 in which he secretly took video recordings of women preparing for a Jewish ritual bath. The sentence was announced by Acting U.S. Attorney Vincent H. Cohen Jr. of the District of Columbia and Chief Cathy L. Lanier of the Metropolitan Police Department (MPD). Freundel pleaded guilty in February 2015 to the misdemeanor charges before the Honorable Senior Judge Geoffrey M. Alprin in the Superior Court of the District of Columbia.\xa0 The sentencing came at the end of a three-hour hearing in which more than a dozen women told the court of the emotional impact they continue to endure because of Freundel’s abuse of his position of trust.\xa0 Judge Alprin sentenced Freundel to 45 days in prison for each of the 52 victims, calli